In [ ]:
import numpy as np
import pandas as pd
import pennylane as qml
from pennylane import numpy as pnp  # Use PennyLane's NumPy for compatibility
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import random
import sys
import warnings
import os
import datetime
from collections import Counter

# Suppress warnings for cleaner output (optional)
warnings.filterwarnings("ignore")

# =============================================================================
# Step 1: Read and Process the Data (Target Sequence)
# =============================================================================

def read_target_sequence(csv_file='your_data.csv', column_name='Probability_Vector'):
    """
    Reads the target sequence from a CSV file and processes it into the required format.

    Parameters:
    - csv_file (str): Path to the CSV file containing your dataset.
    - column_name (str): Name of the column containing the probability vectors.

    Returns:
    - target_sequence (list): List of numpy arrays representing the probability vectors.
    """
    print("Step 1: Reading and processing the data...")
    try:
        data_df = pd.read_csv(csv_file)
        print("Data read from CSV:")
        print(data_df.head())
    except FileNotFoundError:
        print(f"Error: '{csv_file}' not found. Please ensure the file exists in the working directory.")
        sys.exit(1)
    except pd.errors.EmptyDataError:
        print(f"Error: '{csv_file}' is empty.")
        sys.exit(1)
    except pd.errors.ParserError:
        print(f"Error: '{csv_file}' is malformed.")
        sys.exit(1)

    if column_name not in data_df.columns:
        print(f"Error: Column '{column_name}' not found in '{csv_file}'.")
        sys.exit(1)

    # Convert the probability vector column into a list of numpy arrays
    try:
        # Assuming the probability vectors are stored as strings like "[0.1, 0.2, ..., 0.0]"
        target_sequence = data_df[column_name].apply(lambda x: np.array(eval(x))).tolist()
    except Exception as e:
        print(f"Error processing probability vectors: {e}")
        sys.exit(1)

    print(f"Target sequence length: {len(target_sequence)}")
    return target_sequence

# =============================================================================
# Step 2: Define the Quantum System
# =============================================================================

def define_quantum_system(num_qubits):
    """
    Defines the quantum system's device and wires.

    Parameters:
    - num_qubits (int): Number of qubits for the quantum system.

    Returns:
    - dev (Device): PennyLane quantum device.
    """
    print(f"Step 2: Defining quantum system with {num_qubits} qubits...")
    dev = qml.device('default.qubit', wires=num_qubits)
    print("Quantum device initialized.")
    return dev

# =============================================================================
# Step 3: Define the Grover Operator using PennyLane
# =============================================================================

def grover_operator(target_state, num_qubits):
    """
    Constructs the Grover operator for the given target state.

    Parameters:
    - target_state (int): Integer representing the target basis state.
    - num_qubits (int): Number of qubits.

    Returns:
    - grover_op (Operator): Grover operator as a PennyLane operation.
    """
    # Define the oracle
    def oracle():
        # Flip the phase of the target state
        binary_target = format(target_state, '0' + str(num_qubits) + 'b')
        for idx, bit in enumerate(binary_target):
            if bit == '0':
                qml.PauliX(wires=idx)
        qml.MultiControlledX(wires=list(range(num_qubits - 1)) + [num_qubits - 1], control_values='1' * (num_qubits - 1))
        for idx, bit in enumerate(binary_target):
            if bit == '0':
                qml.PauliX(wires=idx)

    # Define the diffusion operator
    def diffusion():
        for qubit in range(num_qubits):
            qml.Hadamard(wires=qubit)
            qml.PauliX(wires=qubit)
        qml.MultiControlledX(wires=list(range(num_qubits - 1)) + [num_qubits - 1], control_values='1' * (num_qubits - 1))
        for qubit in range(num_qubits):
            qml.PauliX(wires=qubit)
            qml.Hadamard(wires=qubit)

    # Combine oracle and diffusion into the Grover operator
    def grover_op():
        oracle()
        diffusion()

    return grover_op

# =============================================================================
# Step 4: Initialize the Random Forest Classifier
# =============================================================================

def initialize_random_forest(n_estimators=100, max_depth=10):
    """
    Initializes the Random Forest classifier for the classical search component.

    Parameters:
    - n_estimators (int): Number of trees in the forest.
    - max_depth (int): Maximum depth of the trees.

    Returns:
    - rf_classifier (RandomForestClassifier): Initialized Random Forest classifier.
    """
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    print("Random Forest classifier initialized.")
    return rf_classifier

# =============================================================================
# Step 5: Initialize the LSTM Model
# =============================================================================

def initialize_lstm(input_shape, output_dim):
    """
    Initializes the LSTM model for handling time steps.

    Parameters:
    - input_shape (tuple): Shape of the input data (timesteps, features).
    - output_dim (int): Dimension of the output layer.

    Returns:
    - lstm_model (Sequential): Compiled LSTM model.
    """
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=False))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print("LSTM model initialized and compiled.")
    return model

# =============================================================================
# Step 6: Assign Fractal Layers to Sequence Positions
# =============================================================================

def assign_fractal_layers(sequence_length, max_layers):
    """
    Assigns fractal layers to each position in the sequence.

    Parameters:
    - sequence_length (int): Length of the target sequence.
    - max_layers (int): Maximum number of fractal layers.

    Returns:
    - fractal_layers (list): List assigning each position to a fractal layer.
    """
    fractal_layers = [0] * sequence_length
    layer = 1
    step = 1
    while layer <= max_layers and step < sequence_length:
        for i in range(0, sequence_length, step * 2):
            if i + step < sequence_length:
                fractal_layers[i + step] = layer
        layer += 1
        step *= 2
    return fractal_layers

# =============================================================================
# Step 7: Synergy Measurements Functions
# =============================================================================

def calculate_synergy_parameters(probs):
    """
    Calculates synergy parameters (alpha, beta, nil) based on measurement probabilities.

    Parameters:
    - probs (numpy.ndarray): Array of measurement probabilities.

    Returns:
    - synergy_params (dict): Dictionary containing synergy parameters and repeat counts.
    """
    # For simplicity, we'll consider alpha as the mean of neighboring probabilities,
    # beta as the standard deviation, and nil as the minimum value among neighbors.

    n = len(probs)
    alphas = np.zeros(n)
    betas = np.zeros(n)
    nils = np.zeros(n)

    # Use circular neighbors for the example (since we have no spatial arrangement)
    for i in range(n):
        neighbors = []
        if i > 0:
            neighbors.append(probs[i - 1])
        if i < n - 1:
            neighbors.append(probs[i + 1])

        if neighbors:
            alphas[i] = np.mean(neighbors)
            betas[i] = np.std(neighbors)
            nils[i] = np.min(neighbors)
        else:
            alphas[i] = probs[i]
            betas[i] = 0
            nils[i] = probs[i]

    synergy_params = {
        'alphas': alphas,
        'betas': betas,
        'nils': nils
    }

    return synergy_params

def update_synergy_counters(synergy_params, synergy_counters):
    """
    Updates synergy counters with the current synergy parameters.

    Parameters:
    - synergy_params (dict): Dictionary containing synergy parameters.
    - synergy_counters (dict): Dictionary of Counter objects for cumulative counts.
    """
    for key in ['alphas', 'betas', 'nils']:
        rounded_values = np.round(synergy_params[key], 4)
        synergy_counters[key].update(rounded_values)

def calculate_synergy_repeats(synergy_params, synergy_counters):
    """
    Calculates repeat counts and cumulative maps for synergy parameters.

    Parameters:
    - synergy_params (dict): Dictionary containing synergy parameters.
    - synergy_counters (dict): Dictionary of Counter objects for cumulative counts.

    Returns:
    - repeats (dict): Dictionary containing repeat counts and maps for each parameter.
    """
    repeats = {}
    for key in ['alphas', 'betas', 'nils']:
        rounded_values = np.round(synergy_params[key], 4)
        cumulative_counts = [synergy_counters[key][val] for val in rounded_values]
        repeat_count = sum(1 for count in cumulative_counts if count > 1)
        repeats[key] = {
            'repeat_count': repeat_count,
            'cumulative_map': cumulative_counts
        }
    return repeats

# =============================================================================
# Step 8: Evolve the Quantum-Classical Hybrid System with Fractal Layers, LSTM, and Synergy Measurements
# =============================================================================

def evolve_system(target_sequence, dev, num_qubits, rf_classifier, lstm_model, fractal_layers, num_runs=10, max_iterations=1000,
                  learning_rate=0.05, fractal_regressive_coeff=0.95, time_steps=5):
    """
    Evolves the quantum-classical hybrid system using Grover’s quantum search, Random Forest, LSTM, Fractal Layers, and Synergy Measurements.

    Parameters:
    - target_sequence (list): List of numpy arrays representing the probability vectors.
    - dev (Device): PennyLane quantum device.
    - num_qubits (int): Number of qubits in the system.
    - rf_classifier (RandomForestClassifier): Initialized Random Forest classifier.
    - lstm_model (Sequential): Initialized LSTM model.
    - fractal_layers (list): List assigning each position to a fractal layer.
    - num_runs (int): Number of independent runs.
    - max_iterations (int): Maximum iterations per run.
    - learning_rate (float): Rate at which the regressive coefficient is adjusted.
    - fractal_regressive_coeff (float): Fixed regressive coefficient for matched positions.
    - time_steps (int): Number of time steps for LSTM input sequence.

    Returns:
    - None
    """
    print("Step 8: Evolving the system iteratively with quantum-classical hybrid, LSTM, fractal layers, and synergy measurements...")

    # Directory to save CSV files
    output_dir = "simulation_results"
    os.makedirs(output_dir, exist_ok=True)

    d = 2 ** num_qubits  # Dimension of the qudit

    # Initialize synergy counters
    synergy_counters = {
        'alphas': Counter(),
        'betas': Counter(),
        'nils': Counter()
    }

    for run in range(1, num_runs + 1):
        print(f"\n--- Run {run} ---")
        iteration = 0
        converged = False

        # Initialize data storage for this run
        run_data = []

        # Initialize a set to track matched positions
        matched_positions = set()

        # Initialize sequence data for LSTM
        lstm_sequence_data = []
        lstm_target_data = []

        while not converged and iteration < max_iterations:
            iteration += 1
            print(f"\nIteration {iteration}")

            # Initialize storage for states
            measurements_over_time = []

            # Iterate over the target sequence
            for idx, target_prob_vector in enumerate(target_sequence):
                # Skip if position's fractal layer is higher than current iteration
                if fractal_layers[idx] > iteration:
                    continue

                # Determine the target state (state with highest probability)
                target_state = np.argmax(target_prob_vector)

                # Define Grover operator for the target state
                grover_op = grover_operator(target_state, num_qubits)

                # Number of Grover iterations (could be adjusted based on synergy measurements)
                num_grover_iterations = 1

                # Define the quantum circuit with dynamic adjustments based on synergy
                @qml.qnode(dev)
                def circuit():
                    # Initialize in uniform superposition
                    for qubit in range(num_qubits):
                        qml.Hadamard(wires=qubit)

                    # Apply Grover iterations
                    for _ in range(num_grover_iterations):
                        grover_op()

                    # Optionally, introduce adjustments based on synergy repeats
                    # For example, apply additional rotations if synergy repeats are low
                    synergy_repeat_counts = synergy_counters['alphas'][target_state]
                    if synergy_repeat_counts < iteration:
                        # Apply an additional rotation to explore more
                        qml.RY(np.pi / 4, wires=0)

                    # Return probabilities
                    return qml.probs(wires=range(num_qubits))

                # Execute the circuit
                probs = circuit()
                measurements_over_time.append(probs)

                # Measured state is the one with the highest probability
                measured_state = np.argmax(probs)

                # Collect data for Random Forest and LSTM
                position_data = {
                    'Run': run,
                    'Iteration': iteration,
                    'Position': idx,
                    'Target_Prob_Vector': target_prob_vector,
                    'Measured_State': measured_state,
                    'Fractal_Layer': fractal_layers[idx],
                }
                # Add measurement probabilities as separate columns
                for state_idx, prob in enumerate(probs):
                    position_data[f'Prob_State_{state_idx}'] = prob

                # Calculate synergy parameters
                synergy_params = calculate_synergy_parameters(probs)
                # Update synergy counters
                update_synergy_counters(synergy_params, synergy_counters)
                # Calculate repeats
                repeats = calculate_synergy_repeats(synergy_params, synergy_counters)

                # Add synergy measurements to position data
                position_data.update({
                    'Alpha_Repeat_Count': repeats['alphas']['repeat_count'],
                    'Beta_Repeat_Count': repeats['betas']['repeat_count'],
                    'Nil_Repeat_Count': repeats['nils']['repeat_count']
                })

                run_data.append(position_data)

                # Prepare data for LSTM (sequence of measurement probabilities)
                lstm_sequence_data.append(probs)
                lstm_target_data.append(target_state)

                # Check if measured state matches target state
                if measured_state == target_state:
                    matched_positions.add(idx)
                    print(f"Matched state at position {idx}: {measured_state}")
                else:
                    print(f"Mismatch at position {idx}: Measured {measured_state} != Target {target_state}")

            # Print progress update
            total_positions = len(target_sequence)
            matched_count = len(matched_positions)
            print(f"\nProgress Update: {matched_count} out of {total_positions} positions have been matched.")

            # Check for convergence (all positions have been assigned and matched)
            if matched_count == total_positions:
                print(f"All positions have been assigned and matched in run {run} after {iteration} iterations!")
                converged = True
                # Save the data to CSV
                df_run = pd.DataFrame(run_data)
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                csv_filename = f"Run_{run}_Data_{timestamp}.csv"
                csv_path = os.path.join(output_dir, csv_filename)
                df_run.to_csv(csv_path, index=False)
                print(f"Data for run {run} saved to '{csv_path}'.")
                break  # Exit the iteration loop

            # Train the Random Forest classifier with accumulated data
            if run_data:
                # Prepare data for training
                df_run = pd.DataFrame(run_data)
                feature_columns = [f'Prob_State_{i}' for i in range(d)] + ['Alpha_Repeat_Count', 'Beta_Repeat_Count', 'Nil_Repeat_Count']
                X_train_rf = df_run[feature_columns]
                y_train_rf = df_run['Target_Prob_Vector'].apply(lambda x: np.argmax(x))

                rf_classifier.fit(X_train_rf, y_train_rf)
                print("Random Forest classifier trained with current data.")

            # Train the LSTM model with accumulated sequence data
            if len(lstm_sequence_data) >= time_steps:
                X_train_lstm = []
                y_train_lstm = []
                for i in range(len(lstm_sequence_data) - time_steps):
                    X_train_lstm.append(lstm_sequence_data[i:i+time_steps])
                    y_train_lstm.append(lstm_target_data[i+time_steps])
                X_train_lstm = np.array(X_train_lstm)
                y_train_lstm = np.array(y_train_lstm)
                y_train_lstm = pd.get_dummies(y_train_lstm).values  # One-hot encoding

                lstm_model.fit(X_train_lstm, y_train_lstm, epochs=1, batch_size=32, verbose=0)
                print("LSTM model trained with current sequence data.")

            # Use LSTM to predict and adjust future iterations
            if len(lstm_sequence_data) >= time_steps:
                latest_sequence = np.array(lstm_sequence_data[-time_steps:]).reshape(1, time_steps, d)
                lstm_prediction = lstm_model.predict(latest_sequence)
                predicted_state_lstm = np.argmax(lstm_prediction, axis=1)[0]
                print(f"LSTM Prediction for next state: {predicted_state_lstm}")

                # Optionally, adjust Grover iterations based on LSTM prediction
                if predicted_state_lstm != target_state:
                    num_grover_iterations += 1  # Increase iterations to enhance amplitude amplification

            # Use Random Forest to predict and adjust future iterations
            if run_data:
                latest_data = run_data[-1]
                latest_features = np.array([latest_data[f'Prob_State_{i}'] for i in range(d)] + [
                    latest_data['Alpha_Repeat_Count'],
                    latest_data['Beta_Repeat_Count'],
                    latest_data['Nil_Repeat_Count']
                ]).reshape(1, -1)
                predicted_state_rf = rf_classifier.predict(latest_features)[0]
                print(f"Random Forest Prediction for next state: {predicted_state_rf}")

                # Optionally, adjust Grover iterations based on RF prediction
                if predicted_state_rf != target_state:
                    num_grover_iterations += 1  # Adjust iterations

        # After all iterations, if not converged, save the data
        if not converged:
            print(f"Run {run} did not fully converge after {max_iterations} iterations.")
            if run_data:
                df_run = pd.DataFrame(run_data)
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                csv_filename = f"Run_{run}_Data_{timestamp}.csv"
                csv_path = os.path.join(output_dir, csv_filename)
                df_run.to_csv(csv_path, index=False)
                print(f"Partial data for run {run} saved to '{csv_path}'.")

        # After each run, plot the measurement results if converged
        if converged and run_data:
            measurements = [row['Measured_State'] for row in run_data]
            target_states = [np.argmax(seq) for seq in target_sequence]

            plt.figure(figsize=(12, 6))
            plt.plot(range(len(measurements)), measurements, label='Measured State', marker='o', markersize=4, linewidth=1)
            plt.plot(range(len(target_states)), target_states, label='Target State', linestyle='--', marker='x', markersize=4)
            plt.xlabel('Position in Sequence')
            plt.ylabel('State')
            plt.title(f'State Measurements Over Time - Run {run}')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        else:
            print(f"No plot generated for run {run}.")

# =============================================================================
# Main Execution
# =============================================================================

def main():
    # Define parameters
    csv_file = 'your_data.csv'              # CSV file containing your dataset
    column_name = 'Probability_Vector'      # Column name in the CSV file
    num_qubits = 5                          # Number of qubits (adjust based on required dimension)
    num_runs = 10                           # Number of independent runs
    max_iterations = 1000                   # Maximum iterations per run
    learning_rate = 0.05                    # Learning rate (not directly used here)
    fractal_regressive_coeff = 0.95         # Fixed regressive coefficient upon successful match
    max_layers = 10                         # Maximum number of fractal layers
    time_steps = 5                          # Number of time steps for LSTM input sequence

    # Step 1: Read the target sequence
    target_sequence = read_target_sequence(csv_file, column_name)

    # Ensure that the probability vectors are normalized
    target_sequence = [vec / np.sum(vec) for vec in target_sequence]

    # Adjust number of qubits based on dimension of the target vectors
    d = len(target_sequence[0])
    num_qubits = int(np.ceil(np.log2(d)))

    # Step 2: Define the quantum system's device
    dev = define_quantum_system(num_qubits)

    # Step 3: (Grover operator is defined within the evolve_system function)

    # Step 4: Initialize the Random Forest classifier
    rf_classifier = initialize_random_forest()

    # Step 5: Initialize the LSTM model
    lstm_input_shape = (time_steps, d)  # (timesteps, features)
    lstm_output_dim = d                 # Output dimension equal to number of states
    lstm_model = initialize_lstm(lstm_input_shape, lstm_output_dim)

    # Step 6: Assign fractal layers to sequence positions
    fractal_layers = assign_fractal_layers(len(target_sequence), max_layers)
    print("Fractal layers assigned to sequence positions.")

    # Step 7: Evolve the quantum-classical hybrid system with fractal layers, LSTM, and synergy measurements
    evolve_system(
        target_sequence=target_sequence,
        dev=dev,
        num_qubits=num_qubits,
        rf_classifier=rf_classifier,
        lstm_model=lstm_model,
        fractal_layers=fractal_layers,
        num_runs=num_runs,
        max_iterations=max_iterations,
        learning_rate=learning_rate,
        fractal_regressive_coeff=fractal_regressive_coeff,
        time_steps=time_steps
    )

# =============================================================================
# Entry Point
# =============================================================================

if __name__ == "__main__":
    main()
